# Power spectrum density analysis of dendritic data. 
This analysis is purely based on locomotory signals bc there is no eeg data available yet.
* 1/27/2024

In [ ]:
import sys, os
import json
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import numpy as np
import scipy.signal as signal
from scipy.interpolate import interp1d

sys.path.append("/home/gergely/code/sleep/src")

from suite2p_class import Suite2p

from imaging_data_class import ImagingData

# from eeg_class import eegData
import behavior_class as bc
import dfof


def freq_calc(data: pd.Series, fs: int = 10):
    """data: npil_eeg data"""
    frequencies, psd = signal.welch(data, fs=fs, detrend="linear")
    return frequencies, psd

In [ ]:
mouse = ImagingData("140502_5")

s2p_folders = mouse.find_suite2p_folders()
for folder in enumerate(s2p_folders):
    print(folder)

In [ ]:
s2p_folder = s2p_folders[8]

mobile_immobile_data = sima_folder = os.path.dirname(s2p_folder)

## Make Sure the folder is set correctly in the `behavior_folders[1][1]` variable!!!
TODO: fix this so that it is not hardcoded

In [ ]:
behavior = bc.behaviorData(mouse_id=mouse.mouse_id)
behavior_folders = [
    (index, folder) for index, folder in enumerate(behavior.find_behavior_folders())
]
print(behavior_folders)
with open(join(behavior_folders[1][1], "mobility_immobility.json"), "rb") as f:
    data = json.load(f)

df = pd.DataFrame(columns=["mob_immob"], data=data)

In [ ]:
df
# True if mobile

In [ ]:
s2p_calc_dfof = dfof.Suite2pDFOF(window=1800)
jia_calc_dfof = dfof.JiaDFOF(slow_trend_window=1800)
s2p_data = Suite2p(s2p_folder=s2p_folder)
s2p_data
cells = s2p_data.get_cells()
npil = s2p_data.get_npil()

In [ ]:
cell_dfof = s2p_calc_dfof.calculate(signal=cells, npil=npil)
npil_dfof = jia_calc_dfof.calculate(signal=npil)

In [ ]:
mean_cellF = cell_dfof.mean(axis=0)
mean_npilF = npil_dfof.mean(axis=0)

In [ ]:
fig, ax = plt.subplots(nrows=3, figsize=(20, 5), sharey=True)
# concatentate the True values:
ax[0].plot(mean_cellF.values, color="k", label="all cells")
ax[0].legend(loc="upper right", labelcolor="k")
ax[1].plot(mean_cellF[df["mob_immob"]].values, label="movement")
ax[1].legend(loc="upper right", labelcolor="k")
ax[2].plot(mean_cellF[~df["mob_immob"]].values, color="orange")
ax[2].legend(["stationary"], loc="upper right", labelcolor="k")
plt.show()

fig.savefig(
    join(sima_folder, "Dendritic dfof movement vs stat.png"),
    bbox_inches="tight",
    dpi=300,
)

### Calculations

In [ ]:
input_data = {
    "mobile_cell": pd.Series(mean_cellF[df["mob_immob"]].values),
    "mobile_npil": pd.Series(mean_npilF[df["mob_immob"]].values),
    "stat_npil": pd.Series(mean_npilF[~df["mob_immob"]].values),
    "stat_cell": pd.Series(mean_cellF[~df["mob_immob"]].values),
}

In [ ]:
results = {}

for key, value in input_data.items():
    freq, psd = freq_calc(value)
    results[key] = {"freq": freq, "psd": psd}

results.keys()

In [ ]:
# Create a figure with 2 subplots in 1 row and 2 columns
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), sharey=True)
fig.suptitle("Power Spectral Density: Awake vs NREM")

# Plot the first semilogy plot in the first subplot
axs[0].semilogy(
    results["mobile_cell"]["freq"], results["mobile_cell"]["psd"], label="mobile"
)
axs[0].semilogy(results["stat_cell"]["freq"], results["stat_cell"]["psd"], label="stat")
axs[0].legend(loc="lower left")

# Plot the second semilogy plot in the first subplot
axs[1].semilogy(
    results["mobile_npil"]["freq"], results["mobile_npil"]["psd"], label="mobile"
)
axs[1].semilogy(results["stat_npil"]["freq"], results["stat_npil"]["psd"], label="stat")
axs[1].legend(loc="lower left")

# Set the title and axis labels for the first subplot
axs[0].set_title("Cell signal")
axs[0].set_xlabel("Frequency (Hz)")
axs[0].set_ylabel("Power/Frequency (dB/Hz)")

# Set the title and axis labels for the second subplot
axs[1].set_title("Npil signal")
axs[1].set_xlabel("Frequency (Hz)")

# Show the plot
plt.show()
fig.savefig(
    join(sima_folder, "PSD plot cell vs npil.png"), bbox_inches="tight", dpi=300
)